In [1]:
import pyro
import congas 
from congas.Interface import Interface
from congas.models import MixtureGaussianEXP, MixtureGaussianNorm, MixtureGaussian
from pyro.infer import SVI, infer_discrete, TraceEnum_ELBO, TraceGraph_ELBO, JitTraceEnum_ELBO
from pyro.optim import ClippedAdam, Adam


data_dict =congas.simulation_data

data_dict['data'] = data_dict['data'] 

interface = Interface()
interface.set_model(MixtureGaussian)
interface.set_loss(TraceEnum_ELBO)
interface.set_optimizer(Adam)

pyro.enable_validation(True)
interface.initialize_model(data_dict)
interface.set_model_params({'K':2})

loss = interface.run(300 ,MAP = True,  param_optimizer={"lr" : 0.01 }, verbose=False, 
                    )



{'K': 2, 'cnv_sd': 0.1, 'theta_scale': 3, 'theta_rate': 1, 'batch_size': None, 'mixture': tensor([0.5000, 0.5000]), 'norm_init_factors': None, 'kmeans': True, 'norm_factor': None, 'assignments': None}
Running MixtureGaussian on 1000 cells for 300 steps
..........

..........

..........

Done!


In [2]:

par: dict = interface.learned_parameters( posterior = False)

nf = par['norm_factor']
assi = par['assignement']

interface.set_model_params({'K':2, 'norm_factor' : nf, 'assignments' : assi})


loss = interface.run(300 ,MAP = False,  param_optimizer={"lr" : 0.01 }, verbose=False, 
                    )

{'K': 2, 'cnv_sd': 0.1, 'theta_scale': 3, 'theta_rate': 1, 'batch_size': None, 'mixture': tensor([0.5000, 0.5000]), 'norm_init_factors': None, 'kmeans': True, 'norm_factor': array([1.6303606 , 1.465912  , 1.8392447 , 1.1223915 , 1.952555  ,
       1.5505942 , 1.6173903 , 2.3700266 , 1.9442044 , 1.9465916 ,
       1.7652797 , 1.4814174 , 1.2380918 , 1.7247386 , 1.3537872 ,
       1.493342  , 1.6126181 , 2.5835366 , 1.7116182 , 1.3681011 ,
       1.0066931 , 1.6591333 , 2.3676364 , 1.2607523 , 1.8189648 ,
       2.6324341 , 1.333512  , 2.075414  , 2.3068097 , 1.502883  ,
       1.2023075 , 1.5863776 , 2.4475536 , 3.3755224 , 1.5851836 ,
       1.4158103 , 2.0372424 , 1.9251248 , 1.9394366 , 1.4229689 ,
       1.5327038 , 1.1784538 , 2.4523227 , 1.4384735 , 2.0944948 ,
       4.4895673 , 1.4969189 , 2.2626731 , 2.830431  , 1.821354  ,
       1.6507852 , 2.9043803 , 1.1677171 , 1.963291  , 1.3585595 ,
       3.5365462 , 2.0420127 , 2.108802  , 2.8709843 , 2.2304685 ,
       2.1744106 , 1.9

/Users/salvatore/miniconda3/envs/ScRNA-CNV-inference/lib/python3.7/site-packages/pyro/infer/traceenum_elbo.py:282: UserWarning: TraceEnum_ELBO found no sample sites configured for enumeration. If you want to enumerate sites, you need to @config_enumerate or set infer={"enumerate": "sequential"} or infer={"enumerate": "parallel"}? If you do not want to enumerate, consider using Trace_ELBO instead.
  warnings.warn('TraceEnum_ELBO found no sample sites configured for enumeration. '


In [5]:
pyro.get_param_store().get_all_param_names()
interface.learned_parameters( posterior = False)





{'param_cnv_probs': array([[1.9670455 , 2.0089116 , 0.9694983 , 1.9858087 , 1.9605836 ,
         1.9828453 , 1.9781042 , 0.9973565 , 1.9946231 , 1.9768931 ,
         0.97358507, 1.978669  , 1.9631417 , 1.9735068 , 2.9864578 ,
         1.9773569 , 1.9372087 , 1.9784609 , 1.9870068 , 2.9234843 ,
         0.98754466, 1.9678982 , 1.9731482 , 1.9858906 , 1.9761035 ,
         2.0026584 , 1.9753547 , 2.003429  , 2.0058928 , 2.005941  ,
         1.0023361 , 1.9584873 ],
        [2.0224593 , 1.9952441 , 1.017173  , 2.0075307 , 2.0201178 ,
         2.0072856 , 1.9779282 , 1.9919523 , 1.9889609 , 1.9830146 ,
         0.9771801 , 0.99366957, 2.0444312 , 1.9978542 , 2.9584427 ,
         1.9734263 , 2.058938  , 1.9604142 , 2.0118475 , 2.9570374 ,
         0.998041  , 1.9482628 , 1.9948382 , 2.0333135 , 1.9761403 ,
         1.9748331 , 1.9898453 , 1.9593898 , 2.0005841 , 2.0432038 ,
         0.98735887, 2.0042555 ]], dtype=float32),
 'param_cnv_var': array([[0.0386754 , 0.03786524, 0.04319578, 0.0348

In [5]:
from importlib import reload  
reload(congas)


<module 'congas' from '/Users/salvatore/PycharmProjects/congas/congas/__init__.py'>